In [1]:
# -*-coding:utf-8 -*-
import os
from __future__ import division
from sympy import *
from sympy import coth
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nptdms import TdmsFile
from more_itertools import chunked
%matplotlib widget

In [2]:
# read raw .tdms file with TdmsFile function
# name example: 20200304-160513 10pN- 30nM DNAp + 0.1mg_mL_BSA #019-002
name = input('please type in the file name:' ) + '.tdms'
# path example: C:\Users\12345\Desktop
path = input('please type in the file path:')
filename = os.path.join(path, name) 
tdms_file = TdmsFile(filename)

please type in the file name: 20190517-141042 test 45 nM labelled  DNAp-snap +2uL trx (RT for 15h) + 625 uM dNTPs #018-001
please type in the file path: C:\Users\KTS260\Google Drive\Work with Matthew\Project2_Watching labelled DNAP replication\20190517-018-pol+exo\force data


In [5]:

time = tdms_file['FD Data']['Time (ms)'][:]
time = np.array(time)
force = tdms_file['FD Data']['Force Channel 0 (pN)'][:]
force = np.array(force)
distance = tdms_file['FD Data']['Distance 1 (um)'][:]
distance = np.array(distance)

# run this step to find out the Time of ROI
plt.figure(figsize=(8,6))
plt.plot(time,force)
plt.xlabel('time/ms')
plt.ylabel('force/pN')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# please enter cycle number of interest
cycle = input('please type in cycle number of interest(e.g.:01):')

# time_from_all = float(input('please type in the starting time of interest:'))
# time_to_all = float(input('please type in the ending time of interest:'))

time_from_exo = float(input('please type in the starting time of exo:'))
time_to_exo = float(input('please type in the ending time of exo:'))

time_from_pol = float(input('please type in the starting time of pol:'))
time_to_pol = float(input('please type in the ending time of pol:'))

please type in cycle number of interest(e.g.:01): 01
please type in the starting time of exo: 7400
please type in the ending time of exo: 96600
please type in the starting time of pol: 105600
please type in the ending time of pol: 289900


In [21]:
# here you will select and analyze your interesting cycle with corresponding time range; 
# define a temporary index to compute time of ROI, and subsuquent ROI of force, distance
# indtemp_all = np.where((time <= time_to_all) & (time >= time_from_all))
indtemp_exo = np.where((time <= time_to_exo) & (time >= time_from_exo))
indtemp_pol = np.where((time <= time_to_pol) & (time >= time_from_pol))

# # all time range of event
# time_range_all = time[indtemp_all]
# force_range_all = force[indtemp_all]
# distance_range_all = distance[indtemp_all]

# exo time range of ROI
time_range_exo = time[indtemp_exo]
force_range_exo = force[indtemp_exo]
distance_range_exo = distance[indtemp_exo]

# pol time range of ROI
time_range_pol = time[indtemp_pol]
force_range_pol = force[indtemp_pol]
distance_range_pol = distance[indtemp_pol]

# all time range of ROI
time_range_all = np.append(time_range_exo, time_range_pol)
force_range_all = np.append(force_range_exo, force_range_pol)
distance_range_all = np.append(distance_range_exo, distance_range_pol)

In [22]:
# parameters for tWLC model: Peter Gross, et al. Nature Physics volume 7, pages731–736(2011)
# dsDNA contour length Lc = 2.85056um; persistent length Lp = 56nm
# the twist rigidity C=440 pN nm2;
# the stretching modulus S=1500 pN;
# the twist–stretch coupling g(F) is given by: g(F) =g0+g1F,where g0=−637 pN nm, g1=17 nm
EEDds,Lc,F,Lp,C,g0,g1,S = symbols('EEDds Lc F Lp C g0 g1 S', real=True)
C = 440
g0= -637
g1 = 17
Lc = 2.85056
Lp = 56
S = 1500
# tWLC model expression:
def tWLC(F):
    EEDds = Lc*(1-0.5*(4.1/(F*Lp))**0.5 + C*F/(-(g0+g1*F)**2 + S*C))
    return (EEDds)

# parameters for FJC model: Smith, S. B., et al. Science 271, 795–799 (1996).
# ssDNA contour length Lss = 4.69504um,
# Kuhn length b = 1.5nm (persistent length is 0.75nm),
# the stretching modulus S=800pN
EEDss,Lss,b,Sss = symbols('EEDss Lss b Sss', real=True)
Lss = 4.69504
b = 1.5
Sss = 800
# FJC model expression:
def FJC(F):
    EEDss = []
    for Fext in F:
        x  = Lss * (coth(Fext * b / 4.1) - 4.1 / (Fext * b)) * (1 + Fext / Sss)
        EEDss.append(x)
    EEDss = np.array(EEDss)
    return (EEDss)

# EEDss,Lss,b,Sss,Fext = symbols('EEDss Lss b Sss Fext', real=True)
# Lss = 4.69504
# b = 1.5
# Sss = 800
# # FJC model expression:
# def FJC(Fext):
#     EEDss  = Lss * (coth(Fext * b / 4.1) - 4.1 / (Fext * b)) * (1 + Fext / Sss)
#     return (EEDss)

In [26]:
Force = np.linspace(0.1,68,1000)

plt.figure(figsize=(8,6))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.xlabel('Distance (um)',fontdict=font)
plt.ylabel('Force(pN)',fontdict=font)
plt.plot(tWLC(Force),Force,color='red', marker='o', linestyle='dashed',linewidth=2, markersize=2,label='tWLC Model')
plt.plot(FJC(Force),Force,color='b', marker='o', linestyle='dashed',linewidth=2, markersize=2,label='FJC Model')

plt.legend()
plt.title('Experimental Data fit to tWLC and FJC Model',fontdict=font)
plt.ylim(-5,72)

plt.show()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
# Default bead size = 1.76um, by running this code to fit the right bead size
bead_size = 1.79

# plot experimental data together with 
plt.plot(distance_range_all - bead_size, force_range_all, marker='o', linestyle='solid',linewidth=2, markersize=2,label='Experimental Data')
plt.legend()
plt.tight_layout()
plt.show()
plt.savefig(path + '-DataFit2Model'+'.png', dpi=150)

End-to-end distance between two optically trapped beads consists ssDNA fraction and dsDNA fraction. By calculating:
- *ssDNA percentage* : ssDNA % =(𝐸𝐸𝐷 −𝑡𝑊𝐿𝐶(𝐹))/(𝐹𝐽𝐶(𝐹)  −𝑡𝑊𝐿𝐶(𝐹))
- *dsDNA percentage* : dsDNA % =1-ssDNA % , 
- *ssDNA/dsDNA junction position* could be obtained following: Junction position =(𝑠𝑠% ∗𝐹𝐽𝐶(𝐹))/(𝑠𝑠% ∗𝐹𝐽𝐶(𝐹)+𝑑𝑠% ∗𝑒𝑊𝐿𝐶(𝐹) ) * EED). 

Junction position changes is then plotted as a function of time.

In [28]:
# Default exo_force is 50pN and pol_force 20pN 
exo_force = 45
pol_force = 20

# calculating the length of dsDNA under exo and pol force
dsDNA_exo_ref = tWLC(exo_force)
dsDNA_pol_ref = tWLC(pol_force)

# calculating the length of ssDNA under exo and pol force
ssDNA_exo_ref = 4.69504 * (coth(exo_force * 1.5 / 4.1) - 4.1 / (exo_force * b)) * (1 + exo_force / 800)
ssDNA_pol_ref =4.69504 * (coth(pol_force * 1.5 / 4.1) - 4.1 / (pol_force * b)) * (1 + pol_force / 800)

In [29]:
# calculating ssDNA under exo and pol force
ssDNA_exo_percentage = (distance_range_exo - bead_size - dsDNA_exo_ref)/(ssDNA_exo_ref - dsDNA_exo_ref)
ssDNA_pol_percentage = (distance_range_pol - bead_size - dsDNA_pol_ref)/(ssDNA_pol_ref - dsDNA_pol_ref)
ssDNA_all_percentage = np.append(ssDNA_exo_percentage,ssDNA_pol_percentage)

# calculating basepairs
# Construct of pkyb1 DNA has a length of 8393bp
basepairs = (1-ssDNA_all_percentage) * 8393

In [51]:
# # calculating ssDNA/dsDNA junction position under exo and pol force
# junction_position_exo = (ssDNA_exo_percentage * ssDNA_exo_ref) * (distance_range_exo - bead_size) / ((ssDNA_exo_percentage * ssDNA_exo_ref) + (1 - ssDNA_exo_percentage) * dsDNA_exo_ref)
# junction_position_pol = (ssDNA_pol_percentage * ssDNA_pol_ref) * (distance_range_pol - bead_size) / ((ssDNA_pol_percentage * ssDNA_pol_ref) + (1 - ssDNA_pol_percentage) * dsDNA_pol_ref)
# junction_position_all = np.append(junction_position_exo, junction_position_pol)

# Under some occasions, DNA polymerase starts from the other side, then we need to ajust the above math expression 
# You need to decide based on the image data
junction_position_exo = (distance_range_exo - bead_size) - (ssDNA_exo_percentage * ssDNA_exo_ref) * (distance_range_exo - bead_size) / ((ssDNA_exo_percentage * ssDNA_exo_ref) + (1 - ssDNA_exo_percentage) * dsDNA_exo_ref)
junction_position_pol = (distance_range_pol - bead_size) - (ssDNA_pol_percentage * ssDNA_pol_ref) * (distance_range_pol - bead_size) / ((ssDNA_pol_percentage * ssDNA_pol_ref) + (1 - ssDNA_pol_percentage) * dsDNA_pol_ref)
junction_position_all = np.append(junction_position_exo, junction_position_pol)

In [64]:
# plot DNA polymerase trace as a function of time
plt.figure(figsize=(8,6))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.title('Time (s)',fontdict=font)
plt.ylabel('Distance(um)',fontdict=font)
plt.scatter(time_range_all/1000,distance_range_all - bead_size,color='black', linestyle='dashed',linewidth=0.1,label='End-to-End Distance')
plt.scatter(time_range_all/1000,junction_position_all,color='green',linewidth=0.1,label='DNA Polymerase Trace')

# plt.fill_between(np.array(time_range_all/1000), np.array(junction_position_all),alpha=0.5)

plt.legend()
plt.ylim(0,4.0)

ax = plt.gca()
ax.invert_yaxis()
ax.xaxis.set_ticks_position('top')


plt.show()
plt.tight_layout()
plt.savefig(path + '-DNApTraces'+'.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
# plot basepair changes as a function of time
plt.figure(figsize=(8,6))
font = {'family': 'serif',
        'weight': 'normal',
        'size': 16}

plt.xlabel('Time (s)',fontdict=font)
plt.ylabel('Basepairs',fontdict=font)
plt.plot(time_range_all/1000,basepairs,color='red', marker='o', linestyle='dashed',linewidth=2, markersize=2,label='Basepairs')

plt.legend()
plt.title('BasePair Changes as a Function of Time',fontdict=font)

plt.show()
plt.tight_layout()
plt.savefig(path + '-BasepairChange'+'.png', dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
# Save all the analyzed data in an excel file
excel_filename = path + '-cycle#' + cycle + "-processedData" + '.xlsx'
# no idea why it causes errors without this step
writer = pd.ExcelWriter(excel_filename)
data = {'time':time_range_all,
        'ssDNA_all_percentage':ssDNA_all_percentage,
        'junction_position_all':junction_position_all,
        'basepairs':basepairs }
df = pd.DataFrame(data)
df.to_excel(writer)
writer.save()